# experiments

> Where the set of experiments and the single experiments are defined.

In [ ]:
#| default_exp experiments

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti

import os
import lir
import numpy as np

from sklearn.base import BaseEstimator
from typing import List, Iterator, Dict, Union, Tuple
from collections import defaultdict

from datetime import datetime

from lir import (LogitCalibrator,
                 NormalizedCalibrator,
                 ELUBbounder,
                 KDECalibrator,
                 FractionCalibrator,
                 IsotonicCalibrator,
                 DummyCalibrator, Xy_to_Xn)

from sql_face.tables import EnfsiImage, EnfsiPair
from lr_video_face.calibration import ScorerModel

## Experiment
One single expleriment defined

In [ ]:
#| export

class Experiment:

    def __init__(self,
        detector: str,
        embeddingModel: str,
        scorer: BaseEstimator,
        calibrator: BaseEstimator,
        calibration_db: List[str],
        enfsi_years: List[int],
        filters: List[str],
        face_image_filters: List[str],
        metrics: str,
        n_calibration_pairs: int,
        embedding_model_as_scorer: bool,
        root_output_dir: str
        ):     
    
    

        self.detector = detector
        self.embeddingModel = embeddingModel
        self.scorer = scorer
        self.calibrator = calibrator
        self.calibration_db = calibration_db
        self.enfsi_years = enfsi_years
        self.filters = filters
        self.face_image_filters = face_image_filters
        self.metrics = metrics
        self.n_calibration_pairs = n_calibration_pairs
        self.embedding_model_as_scorer = embedding_model_as_scorer
        self.root_output_dir = root_output_dir
        self.output_dir = self._get_output_dir()


    def _get_output_dir(self):
        if isinstance(self.calibrator, IsotonicCalibrator):
            calibrator = 'Isotonic Calibrator'
        else:
            calibrator = str(self.calibrator)

        if self.embedding_model_as_scorer:
            folder = f'{self.detector}_{self.embeddingModel}(emb=scorer)_{calibrator.split("(")[0]}'
        else:
            folder = f'{self.detector}_{self.embeddingModel}(emb<>scorer)_{calibrator.split("(")[0]}'

        output_dir = os.path.join(self.root_output_dir, folder)
        return output_dir
    
    
    def perform(self) -> Dict[str, float]:
        """
        Function to run a single experiment with pipeline:
        - Fit model on train data
        - Fit calibrator on calibrator data
        - Evaluate test set
        """

        # Create folder
        self.create_output_dir()
        session = connect_db()

        # Get test pairs per category.
        test_pairs_per_category = self.get_test_pairs_per_category(session)
        # Get calibration pair per category.
        calibration_pairs_per_category = self.get_calibration_pairs_per_category(test_pairs_per_category.keys(),
                                                                                 session)
        '''
        # todo: move later.
        pairs_df = pd.DataFrame(columns=['Image 1', 'Image 2'])
        for category, pairs in calibration_pairs_per_category.items():
            print(category, len(pairs))
            for pair in pairs:
                pairs_df = pairs_df.append({'Image 1': pair.first.croppedImages.images.path, 'Image 2': pair.second.croppedImages.images.path}, ignore_index=True)

        pairs_df.to_excel(f'cal_pairs_{self.filters}_{self.face_image_filters}.xlsx')
        '''

        # Generate lr_system per category.
        lr_systems, test_pairs_per_category = self.generate_lr_systems(calibration_pairs_per_category,
                                                                       test_pairs_per_category, session)

        # Predict LR
        results = self.predict_lr(lr_systems, test_pairs_per_category, session)

        # todo: make necessary variables for graphs.
        return results

   
    def create_output_dir(self):
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)


    def __str__(self):
        """
        Converts the configuration of this experiment to a string that can be
        used to generate file names for example.
        """
        data_values = []
        for k, v in self.data_config.items():
            if k == 'datasets' and isinstance(v, tuple):
                data_values.append('|'.join(map(str, v)))
            else:
                data_values.append(str(v))

        params_str = '_'.join(map(str, self.params.values()))
        return '_'.join(map(str, [
            self.scorer,
            self.calibrator,
            params_str
        ])).replace(':', '-')  # Windows forbids ':'

        return None

## Experimental Setup
The set of all experiments for one scenario.

In [ ]:
#| export

class ExperimentalSetup:
    def __init__(self, 
                detectors, 
                embeddingModels, 
                calibrator_names, 
                calibration_db, 
                enfsi_years, 
                filters, 
                face_image_filters,
                metrics, 
                n_calibration_pairs, 
                embedding_model_as_scorer,
                results_folder:str,
                session,
                name:datetime = datetime.now().strftime("%Y-%m-%d %H %M %S")
                ):

        self.detectors = detectors
        self.embeddingModels = embeddingModels
        self.calibrators = self._get_calibrators(calibrator_names)
        self.calibration_db = calibration_db
        self.enfsi_years = enfsi_years
        self.filters = filters
        self.face_image_filters = face_image_filters
        self.metrics = metrics
        self.n_calibration_pairs = n_calibration_pairs
        self.embedding_model_as_scorer = embedding_model_as_scorer
        self.session = session
        self.name = datetime.now().strftime("%Y-%m-%d %H %M %S")
        self.output_dir= self._make_output_dir(results_folder)
        self.experiments = self.prepare_experiments()
        self.cllr_expert_per_year = self._get_cllr_expert_per_year()


    @staticmethod
    def _get_calibrators(calibrator_names: List[str]) \
            -> List[BaseEstimator]:
        """
        Parses a list of CALIBRATORS configuration names and returns the
        corresponding calibrators. 
        :param calibrator_names: List[str]
        :return: List[ScorerModel]
        """
        CALIBRATORS = {
            'logit': LogitCalibrator(),
            'logit_normalized': NormalizedCalibrator(LogitCalibrator()),
            'KDE': KDECalibrator(),
            'elub_KDE': ELUBbounder(KDECalibrator()),
            'fraction': FractionCalibrator(),
            'isotonic': IsotonicCalibrator(add_misleading=1)
        }  
        # todo: isotonic needs repr method. Could create an instance with one.
        return [CALIBRATORS[c] for c in calibrator_names]

    
    def _get_directory(self):
        filters = self.filters + self.face_image_filters
        subfolder = f'[{",".join(filters)}]'
        folder = f'C_({self.n_calibration_pairs})_[{",".join(self.calibration_db)}]_T_[\
                {",".join([str(year) for year in self.enfsi_years])}]'
        return os.path.join(folder, subfolder)

    def _make_output_dir(self, results_folder):
        directory = self._get_directory()
        output_dir = os.path.join(results_folder, directory)
        if not os.path.exists(results_folder):
            os.makedirs(results_folder)
        return output_dir

    # def __post_init__(self):
    #     self.calibrators = self._get_calibrators(self.calibrator_names)
    #     self.output_dir: str = self.make_output_dir()
    #     

     
    def _get_cllr_expert_per_year(self):
        session = self.session
        expert_eval = session.query(EnfsiImage.year, EnfsiPair.same, EnfsiPair.ExpertsLLR) \
            .filter(EnfsiImage.image_id == EnfsiPair.second_id,
                    EnfsiImage.year.in_(self.enfsi_years)) \
            .all()        

        y_test_per_year = defaultdict(list)
        LLR_experts_per_year = defaultdict(list)

        for year, y, LLR_expert in expert_eval:
            y_test_per_year[year].append(y)
            LLR_experts_per_year[year].append(LLR_expert)

        cllr_experts_per_year = defaultdict(list)

        for year in self.enfsi_years:
            LLR_experts = np.asarray(LLR_experts_per_year[year])
            for i, expert in enumerate(LLR_experts.T):
                # todo: overflow bug with np.power(10, expert) for year 2017.
                LR_expert = np.asarray([np.power(10, i) for i in expert])
                cllr_expert = lir.metrics.cllr(LR_expert,
                                               np.asarray(y_test_per_year[year]))
                cllr_experts_per_year[year].append(cllr_expert)

        return cllr_experts_per_year

    


    def prepare_experiments(self) -> List[Experiment]:
        """
        Returns a list of all experiments that fall under this setup.

        :return: List[Experiment]
        """
        experiments = []
        for detector in self.detectors:
            for embeddingModel in self.embeddingModels:
                for calibrator in self.calibrators:
                    if self.embedding_model_as_scorer:
                        scorer = ScorerModel(metric=self.metrics, embeddingModel=embeddingModel)
                    else:
                        scorer = LogisticRegression(solver='lbfgs')
                    experiments.append(Experiment(
                        detector,
                        embeddingModel,
                        scorer,
                        calibrator,
                        self.calibration_db,
                        self.enfsi_years,
                        self.filters,
                        self.face_image_filters,
                        self.metrics,
                        self.n_calibration_pairs,
                        self.embedding_model_as_scorer,
                        self.output_dir
                    ))

        return experiments

    def __iter__(self) -> Iterator[Experiment]:
        return iter(self.experiments)

    def __len__(self) -> int:
        return len(self.experiments)

   

   

    

    # def count_detected_faces(self, output_dir):
    #     session = connect_db()
    #     all_faces = (session.query(EnfsiImage.year, Detector.name, CroppedImage.face_detected, func.count("*")) \
    #                  .join(EnfsiImage)
    #                  .join(Detector)
    #                  .filter(EnfsiImage.year.in_(self.enfsi_years))
    #                  .group_by(EnfsiImage.year, Detector.name, CroppedImage.face_detected)
    #                  .all()
    #                  )
    #     detected_faces_df = pd.DataFrame.from_records(all_faces)
    #     detected_faces_df.to_excel(os.path.join(output_dir, f'detected_faces.xlsx'))
    #     session.close()

   

    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()